In [63]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import time
import zipfile

In [64]:


# Download CoNLL 2003 Dataset
import os
from pathlib import Path
import urllib.request

#if not Path("eng.train").is_file():
#    print("File Not found will downloading it!")
#    url = "https://github.com/patverga/torch-ner-nlp-from-scratch/raw/master/data/conll2003/eng.train"
#    urllib.request.urlretrieve(url, 'eng.train')
#else:
#    print("File already present.")



In [65]:
import sparknlp 

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  4.1.0
Apache Spark version:  3.2.1


In [66]:
from sparknlp.training import CoNLL
conll = CoNLL()
data = conll.readDataset(spark, path='eng_custom.train')
import pyspark.sql.functions as F

#data.select(F.explode(F.arrays_zip('token.result', 'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("ner_label")).show(truncate=50)

In [67]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [68]:

#embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
#          .setInputCols(["document", "token"])\
#          .setOutputCol("embeddings")
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
         .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(100)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [69]:



ready_data = embeddings.transform(data)

ready_data.show(10)

#ner_model = ner_pipeline.fit(data)
#ner_model.stages[-1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

#embeddings = WordEmbeddingsModel.pretrained()\
#.setOutputCol('embeddings')



+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| text|            document|            sentence|               token|                 pos|               label|          embeddings|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| make|[{document, 0, 3,...|[{document, 0, 3,...|[{token, 0, 3, ma...|[{pos, 0, 3, 0, {...|[{named_entity, 0...|[{word_embeddings...|
|  can|[{document, 0, 2,...|[{document, 0, 2,...|[{token, 0, 2, ca...|[{pos, 0, 2, 0, {...|[{named_entity, 0...|[{word_embeddings...|
|   do|[{document, 0, 1,...|[{document, 0, 1,...|[{token, 0, 1, do...|[{pos, 0, 1, 0, {...|[{named_entity, 0...|[{word_embeddings...|
| read|[{document, 0, 3,...|[{document, 0, 3,...|[{token, 0, 3, re...|[{pos, 0, 3, 0, {...|[{named_entity, 0...|[{word_embeddings...|
|write|[{document, 0, 4,...|[{document, 0, 4,...|[{token, 0, 4

In [70]:
start = time.time()
print("Start fitting")
ner_model = nerTagger.fit(ready_data)
#ner_model = ner_pipeline.fit(ready_data)
print("Fitting has ended")
print (time.time() - start)


Start fitting


2022-10-05 19:57:40.662411: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 7 - chars: 25 - training examples: 131
Epoch 1/100 started, lr: 0.003, dataset size: 131
Epoch 1/100 - 0.71s - loss: 27.80569 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 1	 1.0	 0.9285714	 0.9629629
Act	 1	 0	 15	 1.0	 0.0625	 0.11764706
0	 28	 13	 1	 0.68292683	 0.9655172	 0.8
Obj	 13	 9	 2	 0.59090906	 0.8666667	 0.70270264
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 55 fp: 22 fn: 22 labels: 5
Macro-average	 prec: 0.65476716, rec: 0.5646511, f1: 0.6063793
Micro-average	 prec: 0.71428573, rec: 0.71428573, f1: 0.7142857
Epoch 2/100 started, lr: 0.0029850747, dataset size: 131
Epoch 2/100 - 0.08s - loss: 38.254234 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 0	 0	 1.0	 1.0	 1.0
Act	 16	 33	 0	 0.3265306	 1.0	 

Epoch 17/100 - 0.08s - loss: 0.53847444 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0.7692308
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 67 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.59261435, rec: 0.5902778, f1: 0.5914438
Micro-average	 prec: 0.8701299, rec: 0.8701299, f1: 0.8701299
Epoch 18/100 started, lr: 0.0027649768, dataset size: 131
Epoch 18/100 - 0.08s - loss: 0.2650541 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0

Epoch 32/100 - 0.07s - loss: 0.08492909 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0.7692308
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 67 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.59261435, rec: 0.5902778, f1: 0.5914438
Micro-average	 prec: 0.8701299, rec: 0.8701299, f1: 0.8701299
Epoch 33/100 started, lr: 0.002586207, dataset size: 131
Epoch 33/100 - 0.08s - loss: 0.06767298 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 2	 5	 0.8333333	 0.6666667	 0.

Epoch 47/100 - 0.08s - loss: 0.03398473 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 5	 2	 0.7368421	 0.875	 0.79999995
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 1	 0	 0.96666664	 1.0	 0.9830508
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0.7692308
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 67 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.5909888, rec: 0.5902778, f1: 0.5906331
Micro-average	 prec: 0.8701299, rec: 0.8701299, f1: 0.8701299
Epoch 48/100 started, lr: 0.0024291498, dataset size: 131
Epoch 48/100 - 0.08s - loss: 0.026533343 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 5	 2	 0.7368421	 0.875	 0.79999995
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 1	 0	 0.96666664	 1.0	 0.9830508
Obj	 10	 1	 5	 0.90909094	 0.6666667

Epoch 62/100 - 0.08s - loss: 0.009419935 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0.7692308
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 67 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.59261435, rec: 0.5902778, f1: 0.5914438
Micro-average	 prec: 0.8701299, rec: 0.8701299, f1: 0.8701299
Epoch 63/100 started, lr: 0.0022900763, dataset size: 131
Epoch 63/100 - 0.08s - loss: 0.06708416 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	

Epoch 77/100 - 0.08s - loss: 0.03588545 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0.7692308
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 67 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.59261435, rec: 0.5902778, f1: 0.5914438
Micro-average	 prec: 0.8701299, rec: 0.8701299, f1: 0.8701299
Epoch 78/100 started, lr: 0.002166065, dataset size: 131
Epoch 78/100 - 0.08s - loss: 0.09276403 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 2	 5	 0.8333333	 0.6666667	 0.

Epoch 92/100 - 0.08s - loss: 0.0045556356 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.6666667	 0.7692308
Pos	 0	 0	 3	 0.0	 0.0	 0.0
tp: 67 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.59261435, rec: 0.5902778, f1: 0.5914438
Micro-average	 prec: 0.8701299, rec: 0.8701299, f1: 0.8701299
Epoch 93/100 started, lr: 0.0020547945, dataset size: 131
Epoch 93/100 - 0.08s - loss: 0.011196038 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 14	 1	 0	 0.93333334	 1.0	 0.9655172
Act	 14	 4	 2	 0.7777778	 0.875	 0.8235294
Neg	 0	 2	 0	 0.0	 0.0	 0.0
0	 29	 2	 0	 0.9354839	 1.0	 0.96666664
Obj	 10	 1	 5	 0.90909094	 0.666666

In [71]:
ner_model.write().overwrite().save("./pip_wo_embedd/")
import shutil

shutil.make_archive("/home/sjhjrok/Documents/NLP/pip_wo_embedd", 'zip', "/home/sjhjrok/Documents/NLP/pip_wo_embedd")

2022-10-05 19:57:50.769771: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

'/home/sjhjrok/Documents/NLP/pip_wo_embedd.zip'

In [72]:
text_list = [
    """Navagate to kitchen"""
]